# Semantic search index creation 

### *Guide*

Anton Antonov    
September 2024  

-----

## Introduction

This notebook shows how to create an LLM-computed vector database over the paragraphs of relatively large text.

Here is the Retrieval Augmented Generation (RAG) workflow we consider:

- The document collection is ingested.
- The documents are split into chunks of relevant sizes.
- Large Language Model (LLM) embedding vectors are obtained for all chunks.
- A vector database is created with these embedding vectors and stored locally. Multiple local databases can be created.
- A relevant local database is imported for use.
- An input query is provided to a retrieval system.
- The retrieval system retrieves relevant documents based on the query.
- The top K documents are selected for further processing.
- The model is fine-tuned using the selected documents.
- The fine-tuned model generates an answer based on the query.
- The output answer is presented to the user.

Here is a Mermaid-JS component diagram that shows the components of performing the Retrieval Augmented Generation (RAG) workflow:

```mermaid
flowchart LR
    subgraph LocalVDB[Local Folder]
        A(Vector Database 1)
        B(Vector Database 2)
        C(Vector Database N)
    end
    D[Ingest Vector Database] -.- LocalVDB
    D --> E
    E[/User Query/] --> F[Retrieval]
    F --> G[Document Selection]
    G -->|Top K documents| H(Model Fine-tuning)
    H --> I[[Generation]]
    I <-.-> LLM{{LLM}}
    I -->J[/Output Answer/]
    G -->|Top K passages| K(Model Fine-tuning)
    K --> I
```

In this diagram:

- There are multiple local vector databases that are stored and maintained locally.
- A vector database from the local collection is selected and ingested.
- An input query provided by the user initiates the RAG workflow.
- The workflow then proceeds with: 
  - retrieval
  - document selection
  - model fine-tuning
  - answer generation
  - presenting the final output


-------

## Setup

In [1]:
use Data::Importers;
use LLM::Functions;
use XDG::BaseDirectory :terms;

use LLM::RetrievalAugmentedGeneration;
use LLM::RetrievalAugmentedGeneration::VectorDatabase;

use Data::Reshapers;
use Data::Summarizers;

------

## Ingest text

Ingest the transcript of the (3.5 hours) discussion [CWv1]:

In [2]:
#my $url = 'https://podscripts.co/podcasts/modern-wisdom/833-eric-weinstein-are-we-on-the-brink-of-a-revolution';
my $url = 'https://podscripts.co/podcasts/modern-wisdom/747-eric-weinstein-why-does-the-modern-world-make-no-sense';
my $txtEN = data-import($url, 'plaintext');

text-stats($txtEN)

(chars => 185217 words => 28485 lines => 4580)

Take the "proper transcript" part:

In [3]:
my $txtEN2 = $txtEN.substr($txtEN.index('Starting point is 00:00:00'));
text-stats($txtEN2)

(chars => 182123 words => 28138 lines => 4522)

Split into paragraphs and make the paragraphs compact:

In [4]:
my @paragraphs = $txtEN2.split(/ 'Starting point is' \h+ [\d ** 2]+ % ':' /):g;
@paragraphs .= map({ $_.subst(/\n+/, "\n"):g});
@paragraphs.elems

284

Show a sample of the paragraphs:

In [5]:
#% html
@paragraphs.pick(4) ==> to-html()

like that since primary school since i was 11 or 10. Yeah.
Is there religious music that moves you?
See, you have the major scale as the centerpiece of Western music.
Do you need a guitar?
We're okay.
Wait, do you have a guitar?
We have a guitar.
Can we get the guitar?
 
 talking about Jeffrey Epstein, everybody talking about him,
and spend half of that time saying,
what do we think about whoever was behind Jeffrey Epstein?
Whatever was behind Jeffrey Epstein
is what I think cared about gravity,
cared about space-time, cared about physics.
And you get to use this supposed financier as a wedge
to be able to start to break this open?
 
 is that people tend to ask clustered questions.
And I'm always looking for that interviewer
who's going to ask me things that are just like
people haven't heard um mostly what we do is we just do retreads of the same old
questions and it's not a critique of either one of us as interviewers it's just we don't know how
to get out of our traffic circle
where we go around and around.
I'm trying to build the most exciting thing in the world,
 
 Well, I saw this frustration that lawmakers had because they were getting compartmentalized. If
you don't ask precisely the right person, precisely the right question in precisely the right way,
you're not allowed to get an answer. You don't get an answer. But you couldn't even,
look, if you don't know what a Ramanian manifold is,
if you don't know what a determinant line bundle is, there's no way you can ask intelligent
questions about alien visitation. How did they get here? There are no scientists.
There are no relevant scientists in this story.
Does anybody find that at all odd?

------

## Make vector database

Make an empty vector database object:

In [6]:
my $vdbObj = LLM::RetrievalAugmentedGeneration::VectorDatabase.new(name => 'No747');

VectorDatabase(:id("5cb40fbb-9f69-48ca-9fc1-03ec8059ed99"), :name("No747"), :elements(0), :sources(0))

Make an LLM access specification:

In [7]:
#my $conf = llm-configuration("ChatGPT", model => 'text-embedding-002');
my $conf = llm-configuration("Gemini");

$conf.Hash.elems

24

Create the semantic index for the vector database object (an profile it):

In [8]:
my $tstart = now;
$vdbObj.create-semantic-search-index(@paragraphs, method => 'by-max-tokens', max-tokens => 2048, e => $conf):embed;
my $tend = now;
say "Time to make the semantic search index: {$tend - $tstart} seconds.";

Time to make the semantic search index: 142.989442032 seconds.


By default the vector database object is exported in a sub-directory of [`$XDG_DATA_HOME`](https://specifications.freedesktop.org/basedir-spec/latest/index.html):

In [9]:
# The sub-directory
my $dirname = data-home.Str ~ '/raku/LLM/SemanticSearchIndex';

# The exported vector database base file name
my $basename = "SemSe-{$vdbObj.id}.json";

# Corresponding IO:Path object
my $file = IO::Path.new(:$dirname, :$basename);

# Check for existence
$file.f

True

The export path is saved in the vector database object:

In [10]:
$file.Str eq $vdbObj.location

True

Show a sample of the text chunks:

In [11]:
#% html
$vdbObj.text-chunks.pairs.pick(4).sort(*.key) ==> to-html()

061.0,"Those are just words. There aren't any other routes. Okay, all right. Fair enough. There are words. There are no other routes. There are just words. That is the world's"
076.0,"super smart tyrannical string theorist leader? It's not tyrannical look, I don't know how to say this because I'm obviously a critic i i revere this person this is very painful for me to say you know if you if you ask me of all the people's minds on planet earth that i i revere the wonder that is ed witton's brain is beyond almost anything I can communicate, at least when you have a Beethoven or a, I don't know, an Art Tatum or a Picasso or Modigliano, you can see what it"
079.0,"and whether we can leave Earth. Is there any way we can get access to more energy? Is there any way that we can reveal space-time to not be fundamental so that maybe we can do something that would be confused with going faster than light. Maybe we can reach the stars through methods that we can't understand using what we have. Why is Ed Witten guarding the exit? Ed, there are other theories. There have been theories for 40 years. I met you in your office in 1984, 85 in Princeton on a snowy day. And you threw me out of your office for what reason? Because I started talking to you about the fact that I didn't think you were right"
139.0,"who says, okay, you don't want the truth. We need to have stories. Let's just make up stuff and put stuff in our pockets. How much of it is coordination? How much of it's cowardice? Well, I would rephrase that a little differently. Maybe I would say, uh, nobody smart has gotten anything to work like this in a long time. The reason we have Donald Trumpe biden is that everybody failed i failed i've been podcasting reaching millions i've been teaching people about all sorts of"


Show dimensions and data type of the obtained vectors:

In [12]:
say "dimensions : ", $vdbObj.database.&dimensions;
say "data type  : ", deduce-type($vdbObj.database);

dimensions : (283 768)
data type  : Assoc(Vector(Atom((Str)), 283), Tuple([(Any) => 283], 283), 283)


-------

## References

### Articles

[AA1] Anton Antonov, 
["Outlier detection in a list of numbers"](https://rakuforprediction.wordpress.com/2022/05/29/outlier-detection-in-a-list-of-numbers/),
(2022),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

### Packages

[AAp1] Anton Antonov,
[WWW::OpenAI Raku package](https://github.com/antononcube/Raku-WWW-OpenAI),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[WWW::PaLM Raku package](https://github.com/antononcube/Raku-WWW-PaLM),
(2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[LLM::Functions Raku package](https://github.com/antononcube/Raku-LLM-Functions),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp4] Anton Antonov,
[LLM::Prompts Raku package](https://github.com/antononcube/Raku-LLM-Prompts),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp5] Anton Antonov,
[ML::FindTextualAnswer Raku package](https://github.com/antononcube/Raku-ML-FindTextualAnswer),
(2023-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp6] Anton Antonov,
[Math::Nearest Raku package](https://github.com/antononcube/Raku-Math-Nearest),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp7] Anton Antonov,
[Math::DistanceFunctions Raku package](https://github.com/antononcube/Raku-Math-DistanceFunctions),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp8] Anton Antonov,
[Statistics::OutlierIdentifiers Raku package](https://github.com/antononcube/Raku-Statistics-OutlierIdentifiers),
(2022),
[GitHub/antononcube](https://github.com/antononcube).

## Videos

[CWv1] Chris Williamson,
["Eric Weinstein - Are We On The Brink Of A Revolution? (4K)"](https://www.youtube.com/watch?v=PYRYXhU4kxM),
(2024),
[YouTube/@ChrisWillx](https://www.youtube.com/@ChrisWillx).   
([transcript](https://podscripts.co/podcasts/modern-wisdom/833-eric-weinstein-are-we-on-the-brink-of-a-revolution).)